In [1]:
import sys
import itertools
import logging
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.sql import SparkSession

import json
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes

from pyspark.ml.feature import Tokenizer,StopWordsRemover
from random import randint


# from pycorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP('http://localhost:9000')

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Running Spark Version %s" % (spark.version))

#### load business data ###
# path_business ="yizhan/Desktop/cs181/dataset/buiness.json"
path_business = "dataset/business.json"
df_business_raw = spark.read.json(path_business)
# df_business_raw.first()

businessDF = df_business_raw.select(df_business_raw["business_id"], df_business_raw["name"], df_business_raw["categories"])
print(businessDF.first())
businessDF.printSchema()
businessDF.createOrReplaceTempView("business")

# path_review="yizhan/Desktop/CS181/yelp dataset/review.json"
path_review="dataset/review.json"
df_review_raw = spark.read.json(path_review)
# df_review_raw.first()

reviewDF= df_review_raw.select(df_review_raw["review_id"], df_review_raw["user_id"], df_review_raw["business_id"], df_review_raw["text"])
print(reviewDF.first())
reviewDF.printSchema()
reviewDF.createOrReplaceTempView("review")

# join reviews with busines on businnes_id 
merged_review = spark.sql("SELECT review.business_id, user_id, text, categories, name FROM review join business on review.business_id == business.business_id")

Running Spark Version 2.2.0
Row(business_id='YDf95gJZaq05wvo7hTQbbQ', name='Richmond Town Square', categories=['Shopping', 'Shopping Centers'])
root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)

Row(review_id='VfBHSwC5Vz_pbFluy07i9Q', user_id='cjpdDjZyprfyDG3RlkVG3w', business_id='uYHaNptLzDLoV_JZ_MuzUA', text='My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comforta

In [12]:
merged_review.head()

Row(business_id='--9e1ONYQuAa-CB_Rrw7Tw', user_id='4LxKRRIikhr65GfPDW626w', text="What can I say.. Wowzers! Probably one of the best steak houses I've been too. Service was absolutely flawless and dinner was excellent . Ordered seafood tower, wedge, wagyu filet, chateaubriand, bacon grits and sautéed  mushrooms Will definitely be back!", categories=['Steakhouses', 'Cajun/Creole', 'Restaurants'], name='Delmonico Steakhouse')

In [13]:
merged_review.count()

4736897

In [5]:
merged_review.createOrReplaceTempView("merged_review")
distinct_cat_list = spark.sql("SELECT DISTINCT merged_review.categories, count(*) as num FROM merged_review GROUP BY categories ORDER BY num DESC")

In [6]:
distinct_cat_list.count()

67748

In [7]:
distinct_cat_list.head(10)

[Row(categories=['Mexican', 'Restaurants'], num=49696),
 Row(categories=['Restaurants', 'Mexican'], num=48693),
 Row(categories=['Chinese', 'Restaurants'], num=26761),
 Row(categories=['Restaurants', 'Italian'], num=25753),
 Row(categories=['Pizza', 'Restaurants'], num=25664),
 Row(categories=['Restaurants', 'Chinese'], num=25256),
 Row(categories=['Restaurants', 'Pizza'], num=24561),
 Row(categories=['Food', 'Coffee & Tea'], num=23617),
 Row(categories=['Italian', 'Restaurants'], num=20377),
 Row(categories=['Coffee & Tea', 'Food'], num=19923)]

In [25]:
# sample = merged_review.rdd.take(30000)
sample = merged_review.rdd.take(100000)
# sample = merged_review.rdd

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57690)
Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:57690)

In [22]:
metadata = merged_review.rdd

# Data Preprocessing 

In [20]:
iteratorlist = metadata.collect()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57690)
Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:57690)

In [13]:
list = []
label_to_num_dict = {}
num_to_label_dict = {}
index = 0 
for line in iteratorlist:
    labels = line["categories"]
    if len(labels) <= 1:
        continue
    # print(labels)
    text = line["text"]
    # print(text)
    
    append = False
    for label in labels:
        if label == 'Restaurants':
            append = True
    
    if append == True:       
        label_index_to_choose = randint(0, len(labels)-1)
        label = labels[label_index_to_choose]
        while label == 'Restaurants'or label == 'Food':
            label_index_to_choose = randint(0, len(labels)-1)
            label = labels[label_index_to_choose]
            
        if label not in label_to_num_dict:
            # print(index, " represents ",label)
            label_to_num_dict[label] = index
            num_to_label_dict[index] = label 
            index += 1  
            
        # covert the category label to numerical value        
        num_label = label_to_num_dict[label]
        list.append([num_label,text,labels, line['business_id']])        

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 29 in stage 26.0 failed 1 times, most recent failure: Lost task 29.0 in stage 26.0 (TID 1259, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writ

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58884)
Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/Wenonah/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/Wenonah/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/Wenonah/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
---------------

In [64]:
# processing on sample 
list = []
label_to_num_dict = {}
num_to_label_dict = {}
index = 0 
#iteratorlist = sample.collect()
#for line in iteratorlist:
for line in sample:
    labels = line["categories"]
    if len(labels) <= 1:
        continue
    # print(labels)
    text = line["text"]
    # print(text)
    
    append = False
    for label in labels:
        if label == 'Restaurants':
            append = True
    
    if append == True:       
        label_index_to_choose = randint(0, len(labels)-1)
        label = labels[label_index_to_choose]
        while label == 'Restaurants'or label == 'Food':
            label_index_to_choose = randint(0, len(labels)-1)
            label = labels[label_index_to_choose]
            
        if label not in label_to_num_dict:
            # print(index, " represents ",label)
            label_to_num_dict[label] = index
            num_to_label_dict[index] = label 
            index += 1  
                
        num_label = label_to_num_dict[label] # covert the category label to numerical value
        list.append([num_label,text,labels, line['business_id']])        
        '''
    
        for label in labels:
            if label == 'Restaurants'or label == 'Food':
                continue
            if label not in label_to_num_dict:
                # print(index, " represents ",label)
                label_to_num_dict[label] = index
                num_to_label_dict[index] = label 
                index += 1  
            # covert the category label to numerical value
            num_label = label_to_num_dict[label] 
            list.append([num_label,text])
            break
        '''


In [ ]:
sample_with_class = merged_review.rdd.map(lambda item : generateClass(item, label_to_num_dict, num_to_label_dict, index)).filter(lambda x : len(x) > 0)
# sample_with_class = spark.sparkContext.parallelize(sample).map(lambda item : generateClass(item, label_to_num_dict, num_to_label_dict, index)).filter(lambda x : len(x) > 0)

In [ ]:
sample_list = sample_with_class.collect()

In [ ]:
sample_with_class.count()

In [22]:
print(len(list))

64005


In [65]:
for k, v in label_to_num_dict.items():
    print(k, v)

Steakhouses 0
Cajun/Creole 1
Burgers 2
Fast Food 3
Chinese 4
Mexican 5
French 6
German 7
Bars 8
Wine Bars 9
Nightlife 10
Pizza 11
American (New) 12
Grocery 13
Delis 14
Bakeries 15
Caterers 16
Event Planning & Services 17
Specialty Food 18
Hot Dogs 19
Barbeque 20
Vietnamese 21
Cafes 22
Korean 23
Japanese 24
Asian Fusion 25
Thai 26
Caribbean 27
Chicken Wings 28
Sports Bars 29
American (Traditional) 30
Comfort Food 31
Desserts 32
Italian 33
Bagels 34
Sandwiches 35
Breakfast & Brunch 36
Diners 37
Hotels 38
Bed & Breakfast 39
Hotels & Travel 40
Bistros 41
Coffee & Tea 42
Seafood 43
Salad 44
Fondue 45
Gluten-Free 46
Vegetarian 47
Cocktail Bars 48
Pubs 49
British 50
Scottish 51
Southern 52
Soup 53
Wraps 54
Sushi Bars 55
Mediterranean 56
Portuguese 57
Indian 58
Hawaiian 59
Seafood Markets 60
Meat Shops 61
Gastropubs 62
Juice Bars & Smoothies 63
Himalayan/Nepalese 64
Donuts 65
Dive Bars 66
Kebab 67
Tapas/Small Plates 68
Tapas Bars 69
Beer 70
Wine & Spirits 71
Lounges 72
Brasseries 73
Irish 74
B

# Pipeline

In [146]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# convert to rdd 
sample_with_dup = spark.sparkContext.parallelize(list)
label = sample_with_dup.map(lambda item: item[0])
text = sample_with_dup.map(lambda item: item[1])
original_label = sample_with_dup.map(lambda item: item[2])

schemaString = "label text org_label buz_id"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

sampleDF = spark.createDataFrame(sample_with_dup, schema)
sampleDF.printSchema()

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(sampleDF)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=30)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

sampleDF.printSchema()


root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- org_label: string (nullable = true)
 |-- buz_id: string (nullable = true)

root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- org_label: string (nullable = true)
 |-- buz_id: string (nullable = true)



In [147]:
sampleDF.createOrReplaceTempView("sample")
show_count = spark.sql("SELECT DISTINCT sample.label, count(*) as num FROM sample GROUP BY label")
count_dict = {}
print('label count')
for pair in show_count.collect():
    label = num_to_label_dict[ int(pair.label) ]
    if pair.num > 200:
        count_dict[int(pair.label)] = pair.num
    print(label, pair.num)

label count
Australian 74
German 49
Scottish 15
Tea Rooms 7
Appliances & Repair 5
Bakeries 249
Wraps 28
Jazz & Blues 40
Cooking Classes 3
Local Flavor 5
Pizza 2715
Pakistani 117
Internet Cafes 7
Sports Bars 289
Tapas Bars 376
Coffee & Tea 458
Food Trucks 46
Brasseries 18
Venues & Event Spaces 39
Himalayan/Nepalese 38
Fast Food 841
American (Traditional) 2769
Latin American 35
Bagels 70
Gift Shops 1
Hawaiian 194
Chicken Shop 24
Egyptian 2
Books 1
Bars 2950
Live/Raw Food 3
Cafes 701
Chicken Wings 495
Indonesian 75
Canadian (New) 263
Caterers 282
Sandwiches 1530
Southern 366
Singaporean 11
Karaoke 60
Steakhouses 2936
Chocolatiers & Shops 10
Wine & Spirits 207
Sri Lankan 4
Poutineries 19
Vegetarian 382
Tex-Mex 262
Noodles 159
Fruits & Veggies 3
Curry Sausage 9
Arabian 13
Seafood 779
Mexican 4993
Comfort Food 235
Soul Food 3
Tacos 24
Specialty Food 508
Beer 209
Cinema 15
Appliances 6
Caribbean 83
Meat Shops 286
Butcher 1
Dance Clubs 5
Event Planning & Services 312
Street Vendors 78
Arts & C

In [148]:
for k,v in count_dict.items():
    print(num_to_label_dict[k],v)

print('The total number of classes: ', len(count_dict))

Bakeries 249
Pizza 2715
Sports Bars 289
Tapas Bars 376
Coffee & Tea 458
Fast Food 841
American (Traditional) 2769
Bars 2950
Cafes 701
Chicken Wings 495
Canadian (New) 263
Caterers 282
Sandwiches 1530
Southern 366
Steakhouses 2936
Wine & Spirits 207
Vegetarian 382
Tex-Mex 262
Seafood 779
Mexican 4993
Comfort Food 235
Specialty Food 508
Beer 209
Meat Shops 286
Event Planning & Services 312
Thai 921
Dim Sum 400
French 207
Tapas/Small Plates 410
Korean 383
Sushi Bars 1709
Asian Fusion 1676
Salad 749
Soup 349
Buffets 4032
Greek 231
Indian 441
Italian 1899
Cocktail Bars 252
Middle Eastern 242
Wine Bars 603
Japanese 1651
Desserts 477
Cajun/Creole 714
Barbeque 930
Filipino 220
Mediterranean 429
Breakfast & Brunch 2307
Nightlife 3030
Diners 567
Pubs 372
Chinese 2224
Gastropubs 318
American (New) 3256
Delis 357
Vietnamese 762
Lounges 214
Burgers 1638
The total number of classes:  58


In [ ]:
# display the generated tfidf 
print("tfidf:")
count = 1
for each in rescaledData.collect():    
    print(each)
    count += 1
    if count > 1:
        break

rescaledData.select("label", "features").show()

tfidf:


# Naive Bayes Classifer 

In [69]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml import linalg as ml_linalg
import numpy


In [74]:
def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

data_size = 59581        
def sampleSizeControl(item):
    label = int(item.label) 
    if label in count_dict:
        rand = numpy.random.uniform(low=0.0, high=1.0)
        if rand < 200/count_dict[label]:
            return True
    return False

def naive_sampleSizeControl(item):
    label = int(item.label)
    return label in count_dict

def getTwoClass(item,label1,label2):
    label = num_to_label_dict[int(item.label)]
    if label == label1 or label == label2: 
        return True
    return False
    
def getLabeledPoints(item):
    label = item.label
    features = as_mllib(item.features)
    return LabeledPoint(label,features)

In [ ]:
# data = rescaledData.rdd.filter(lambda item: sampleSizeControl(item)).map(lambda item : getLabeledPoints(item))
data = rescaledData.rdd.filter(lambda item: sampleSizeControl(item))

training, test = data.randomSplit([0.6, 0.4])

model = NaiveBayes.train(training.map(lambda item : getLabeledPoints(item)), 1.0)

# Make prediction and test accuracy.
predictionAndLabel = test.map(lambda p: [model.predict(as_mllib(p.features)), p.org_label, p.label, p.buz_id])
accuracy = 1.0 * predictionAndLabel.filter(lambda item: num_to_label_dict[item[0]] in item[1]).count() / test.count()

# Save and load model
# model.save(sc, "target/tmp/myNaiveBayesModel")
# sameModel = NaiveBayesModel.load(sc, "target/tmp/myNaiveBayesModel")

In [77]:
label1,label2 = 'Steakhouses','American (New)'

In [78]:
data_2_class = rescaledData.rdd.filter(lambda item: getTwoClass(item,label1,label2)).map(lambda item : getLabeledPoints(item))

training_2_class, test_2_class = data_2_class.randomSplit([0.6, 0.4])

model_2_class = NaiveBayes.train(training_2_class, 1.0)

# Make prediction and test accuracy.
predictionAndLabel_2_class = test_2_class.map(lambda p: [model_2_class.predict(p.features), p.label])
accuracy_2_class = 1.0 * predictionAndLabel_2_class.filter(lambda item: item[0] == item[1]).count() / test_2_class.count()

In [72]:
data.count()

11106

In [64]:
data_2_class.count()

6148

# Performance Accessment
accuracy when normalized the sample balance 

In [ ]:
print(accuracy)

In [ ]:
buz_number = predictionAndLabel.map(lambda item : item[3]).distinct().count()

In [137]:
# vote = predictionAndLabel.map(lambda item : (item[3], [item[0],item[1]])).reduceByKey(lambda v1, v2:(v1[1],[v1[0]].append(v2[0])))
#.countbyKey().sort() 
vote = predictionAndLabel.map(lambda item : (item[3], 1)).reduceByKey(lambda v1, v2: v1 + v2).filter(lambda item: item(1) >= 10)

In [138]:
vote.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 194.0 failed 1 times, most recent failure: Lost task 0.0 in stage 194.0 (TID 999, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/Wenonah/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-137-0e40c56ed42f>", line 3, in <lambda>
TypeError: 'tuple' objec

In [111]:
def mode(my_list):
    # Form a new list with the unique elements
    unique_list = sorted(list(set(my_list)))
    # Create a comprehensive dictionary with the uniques and their count
    appearence = {a:my_list.count(a) for a in unique_dist} 
    # Calculate max number of appearences
    max_app = max(appearence.values())
    # Return the elements of the dictionary that appear that # of times
    # return {k: v for k, v in appearence.items() if v == max_app}
    for k, v in appearence.items():
        if v == max_app:
            return k

In [55]:
training.count()

6728

In [102]:
majority_vote_accuracy= vote.filter(lambda item : 'Bars' in item[1]).count()/buz_number
# num_to_label_dict[int(mode(item[0]))]

In [103]:
print(majority_vote_accuracy)

0.0


In [91]:
print(buz_number)

611


In [65]:
print(accuracy_2_class)

0.625556004852406


In [ ]:
for item in predictionAndLabel.collect():
    if item[0] != item[1]:
        prediction = num_to_label_dict[item[0]]
        expected = num_to_label_dict[item[1]]
        print("prediction:", prediction, "expected label: ", expected)

In [62]:
for item in predictionAndLabel.collect():
    prediction = num_to_label_dict[item[0]]
    if prediction not in item[1]:     
        expected = item[1]
        print("prediction:", prediction, " selected label: ", num_to_label_dict[int(item[2])], " expected label: ", expected)

prediction: Meat Shops  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Buffets  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Tapas Bars  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Gastropubs  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Sports Bars  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Filipino  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Bakeries  selected label:  Steakhouses  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Diners  selected label:  Steakhouses  expected label:  [Steakhouses, Cajun/Creole, Restaurants]
prediction: Barbeque  selected label:  Cajun/Creole  expected label:  [Steakhouses, Cajun/Creole, Restaurant

prediction: Caterers  selected label:  Gastropubs  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Tapas/Small Plates  selected label:  Meat Shops  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Wine Bars  selected label:  Specialty Food  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Filipino  selected label:  Gastropubs  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Tapas Bars  selected label:  Gastropubs  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Filipino  selected label:  Meat Shops  expected label:  [Food, Meat Shops, Restaurants, Steakhouses, Specialty Food, Gastropubs, American (New)]
prediction: Dim Sum  selected label:  Mea

prediction: Korean  selected label:  Japanese  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Korean  selected label:  Japanese  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Comfort Food  selected label:  Pubs  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Canadian (New)  selected label:  Tapas/Small Plates  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Meat Shops  selected label:  Tapas/Small Plates  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Italian  selected label:  Tapas/Small Plates  expected label:  [Pubs, Japanese, Restaurants, Bars, Nightlife, Tapas Bars, Tapas/Small Plates]
prediction: Asian Fusion  selected label:  Tapas Bars  expected label:  [Pubs, Japanese, Restaura

prediction: Pubs  selected label:  Lounges  expected label:  [Lounges, Nightlife, Bars, Restaurants, Italian]
prediction: Sandwiches  selected label:  Italian  expected label:  [Lounges, Nightlife, Bars, Restaurants, Italian]
prediction: Comfort Food  selected label:  Lounges  expected label:  [Lounges, Nightlife, Bars, Restaurants, Italian]
prediction: Steakhouses  selected label:  Lounges  expected label:  [Lounges, Nightlife, Bars, Restaurants, Italian]
prediction: Thai  selected label:  Lounges  expected label:  [Lounges, Nightlife, Bars, Restaurants, Italian]
prediction: Cajun/Creole  selected label:  Tex-Mex  expected label:  [Restaurants, Tex-Mex]
prediction: Filipino  selected label:  Tex-Mex  expected label:  [Restaurants, Tex-Mex]
prediction: Japanese  selected label:  Tex-Mex  expected label:  [Restaurants, Tex-Mex]
prediction: Thai  selected label:  Greek  expected label:  [Mediterranean, Sandwiches, Italian, Greek, Salad, Restaurants, Pizza]
prediction: Tapas/Small Plates 

prediction: Greek  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Cajun/Creole  selected label:  Indian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Cajun/Creole  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Coffee & Tea  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Filipino  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Buffets  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Korean  selected label:  Vegetarian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Greek  selected label:  Indian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Sandwiches  selected label:  Indian  expected label:  [Restaurants, Vegetarian, Indian]
prediction: Wine Bars  selected label:  Indian  expected label:  [Restaurants, Vegetarian

...

prediction: Thai  selected label:  French  expected label:  [Restaurants, French]
prediction: Mediterranean  selected label:  French  expected label:  [Restaurants, French]
prediction: Korean  selected label:  French  expected label:  [Restaurants, French]
prediction: Soup  selected label:  French  expected label:  [Restaurants, French]
prediction: Korean  selected label:  French  expected label:  [Restaurants, French]
prediction: Caterers  selected label:  French  expected label:  [Restaurants, French]
prediction: Korean  selected label:  French  expected label:  [Restaurants, French]
prediction: Seafood  selected label:  French  expected label:  [Restaurants, French]
prediction: Desserts  selected label:  French  expected label:  [Restaurants, French]
prediction: Korean  selected label:  French  expected label:  [Restaurants, French]
prediction: Buffets  selected label:  Caterers  expected label:  [Arts & Entertainment, Caterers, Event Planning & Services, Restaurants, American (Trad

prediction: Salad  selected label:  Greek  expected label:  [Italian, Mediterranean, Greek, Restaurants]
prediction: Burgers  selected label:  Greek  expected label:  [Italian, Mediterranean, Greek, Restaurants]
prediction: Sandwiches  selected label:  Greek  expected label:  [Italian, Mediterranean, Greek, Restaurants]
prediction: Thai  selected label:  American (Traditional)  expected label:  [Nightlife, Bars, Steakhouses, Beer Bar, American (Traditional), Restaurants]
prediction: Buffets  selected label:  Thai  expected label:  [Restaurants, Thai]
prediction: Pizza  selected label:  Thai  expected label:  [Restaurants, Thai]
prediction: Mediterranean  selected label:  Thai  expected label:  [Restaurants, Thai]
prediction: Tapas/Small Plates  selected label:  Thai  expected label:  [Restaurants, Thai]
prediction: Nightlife  selected label:  Thai  expected label:  [Restaurants, Thai]
prediction: Gastropubs  selected label:  Korean  expected label:  [Restaurants, Korean]
prediction: Vi

prediction: Tex-Mex  selected label:  Italian  expected label:  [Restaurants, Hot Dogs, Italian, Tapas/Small Plates]
prediction: Thai  selected label:  Tapas/Small Plates  expected label:  [Restaurants, Hot Dogs, Italian, Tapas/Small Plates]
prediction: Caterers  selected label:  Tapas/Small Plates  expected label:  [Restaurants, Hot Dogs, Italian, Tapas/Small Plates]
prediction: Greek  selected label:  Tapas/Small Plates  expected label:  [Restaurants, Hot Dogs, Italian, Tapas/Small Plates]
prediction: Sports Bars  selected label:  Pizza  expected label:  [Restaurants, Pizza]
prediction: Sandwiches  selected label:  Pizza  expected label:  [Restaurants, Pizza]
prediction: Cajun/Creole  selected label:  Pubs  expected label:  [Canadian (New), Food, Pubs, Bars, Gastropubs, Nightlife, Restaurants]
prediction: Mexican  selected label:  Pubs  expected label:  [Canadian (New), Food, Pubs, Bars, Gastropubs, Nightlife, Restaurants]
prediction: French  selected label:  Gastropubs  expected lab

prediction: Sandwiches  selected label:  Delis  expected label:  [Coffee & Tea, Juice Bars & Smoothies, Delis, Food, Restaurants]
prediction: Indian  selected label:  Barbeque  expected label:  [Restaurants, Barbeque]
prediction: Japanese  selected label:  Mexican  expected label:  [Restaurants, Mexican]
prediction: Korean  selected label:  Mexican  expected label:  [Restaurants, Mexican]
prediction: Burgers  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Wine & Spirits  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Steakhouses  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Fast Food  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Tapas Bars  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Mediterranean  selected label:  Barbeque  expected label:  [Barbeque, Restaurants]
prediction: Cajun/Creole  selected label:  Vegetaria

prediction: Vegetarian  selected label:  Bars  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Sandwiches  selected label:  Dim Sum  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Japanese  selected label:  Dim Sum  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Tapas Bars  selected label:  Dim Sum  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Burgers  selected label:  Dim Sum  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Sushi Bars  selected label:  Dim Sum  expected label:  [Nightlife, Restaurants, Breakfast & Brunch, Asian Fusion, Dim Sum, Bars, Chinese]
prediction: Sushi Bars  selected label:  Breakfast & Brunch  expected label:  [Nightlife, Restaurants, Break

In [ ]:
for item in predictionAndLabel_2_class.collect():
    if item[0] != item[1]:
        prediction = num_to_label_dict[item[0]]
        expected = num_to_label_dict[item[1]]
        print("prediction:", prediction, "expected label: ", expected)

# Report

'Dim Sum','Pubs' 
     0.68 accuracy
     only 300 data for each 


'Breakfast & Brunch','American (Traditional)' 
    0.57  accuracy
    5152 total data 

'Italian','Sushi Bars'
    0.694
    3581 total data 

'Pizza', 'Chinese'
    0.6653
    4867 total data

'Thai','Bars'
    0.7705
    3820 total data but imbalance in data 

'American (New)', 'Amercian (Tradition)'
    0.5737
    6013 total data ... pretty good...?

# Levergaed Accuracy Measurement 

In [17]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [20]:
sys.meta_path.append(NotebookFinder())
ls nbpackage
import nbpackage.category_analysis

SyntaxError: invalid syntax (<ipython-input-20-ef000de3822c>, line 2)